In [8]:
import pandas as pd
import geopandas as gpd
import requests, json
from shapely.geometry import Point

In [22]:
blocs = gpd.read_file("./shapes/cleaned_blocs.geojson") 
blocs.crs = {'init': 'epsg:4326'}

blocsdf = blocs.dropna()

In [39]:
blocsdf.shape

(1244, 13)

In [34]:
blocsdf_clean = blocsdf[(blocsdf.geometry.type == 'Polygon') | (blocsdf.geometry.type == 'Multipolygon') ]

In [38]:
blocsdf_clean.shape

(1218, 13)

In [17]:
blocsdf_clean.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,39,097,040400,1,390970404001,Block Group 1,G5030,S,61100938,47314,+39.9734723,-083.3173137,"POLYGON ((-83.37306 39.97948, -83.37195 39.979..."
1,39,097,041300,2,390970413002,Block Group 2,G5030,S,148489592,0,+39.7489452,-083.4261524,"POLYGON ((-83.42445 39.79902, -83.42404 39.798..."
2,39,097,040400,2,390970404002,Block Group 2,G5030,S,7505551,15979,+39.9654940,-083.2688875,"POLYGON ((-83.28836 39.94345, -83.28835 39.943..."
3,39,097,041100,4,390970411004,Block Group 4,G5030,S,79772655,456949,+39.8691662,-083.3286931,"POLYGON ((-83.43864 39.88334, -83.43830 39.883..."
4,39,097,040202,1,390970402021,Block Group 1,G5030,S,127181808,59315,+40.0040443,-083.4289544,"POLYGON ((-83.49854 40.00018, -83.49759 40.000..."


In [40]:
jobs = pd.read_csv('./data/jobs.csv')
# Attempt #2
geom = jobs.apply(lambda x : Point([x['lon'],x['lat']]), axis=1)
gdf = gpd.GeoDataFrame(jobs, geometry=geom)
gdf.crs = {'init': 'epsg:4326'}
gdf.head()
jobdf = gdf.dropna()

In [41]:
jobdf.head()

,id,status,noCharge,completedOn,invoice_total,lon,lat,zip,geometry
1,28460822,Completed,False,2019-07-19T19:15:00,0.0,-83.000895,40.019350,43202,POINT (-83.00089 40.01935)
2,29543583,Canceled,False,2019-07-19T14:45:22.7485338,0.0,-83.014422,40.053505,43214,POINT (-83.01442 40.05351)
4,31264388,Completed,True,2019-06-26T13:36:00,0.0,-83.136459,40.154421,43017,POINT (-83.13646 40.15442)
5,31052042,Completed,True,2019-07-17T14:53:00,0.0,-83.019600,40.040534,43214,POINT (-83.01960 40.04053)
6,31292052,Completed,False,2019-06-26T18:59:00,334.0,-82.803314,39.965785,43068,POINT (-82.80331 39.96578)


In [42]:
merged = gpd.sjoin(jobdf, blocsdf_clean, how="inner", op="within")

In [43]:
merged.head()

,id,status,noCharge,completedOn,invoice_total,lon,lat,zip,geometry,index_right,...,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
1,28460822,Completed,False,2019-07-19T19:15:00,0.0,-83.000895,40.019350,43202,POINT (-83.00089 40.01935),1125,...,000500,2,390490005002,Block Group 2,G5030,S,420268,4544,+40.0207167,-083.0093302
1691,31802427,Completed,False,2019-07-10T15:26:00,89.0,-83.009045,40.021300,43202,POINT (-83.00905 40.02130),1125,...,000500,2,390490005002,Block Group 2,G5030,S,420268,4544,+40.0207167,-083.0093302
2241,31911971,Completed,False,2019-07-18T14:14:00,144.0,-83.009045,40.021300,43202,POINT (-83.00905 40.02130),1125,...,000500,2,390490005002,Block Group 2,G5030,S,420268,4544,+40.0207167,-083.0093302
5000,32665612,Completed,False,2019-09-10T18:53:00,4538.0,-83.011064,40.020007,43202,POINT (-83.01106 40.02001),1125,...,000500,2,390490005002,Block Group 2,G5030,S,420268,4544,+40.0207167,-083.0093302
5265,32616557,Completed,True,2019-09-04T18:38:00,0.0,-83.011064,40.020007,43202,POINT (-83.01106 40.02001),1125,...,000500,2,390490005002,Block Group 2,G5030,S,420268,4544,+40.0207167,-083.0093302


In [44]:
merged.shape

(6393, 22)